<a href="https://colab.research.google.com/github/ToshiroHJJZ/Detection-and-Classification-of-Linguistic-Features-for-Effective-Spam-Identification/blob/main/Original_Spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Install necessary libraries
!pip install textattack
!pip install transformers # install the transformers library

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 27.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 60.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import textattack
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import os

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
# Load data

# encountered a utf-8 error,UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 606-607:
#invalid continuation byte, encoding pandas to use 'latin-1', or 'ISO-8859-1'
df = pd.read_csv("/content/drive/MyDrive/spam.csv", encoding='latin-1')

In [8]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [9]:
#drop empty columns
df=df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)

In [10]:
x=df['v2']
y=df['v1']

In [11]:
# label transform v1
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(y)

In [12]:
y[:5]

array([0, 0, 1, 0, 0])

##Aversarial Attack

In [13]:
from IPython import get_ipython
from IPython.display import display

In [15]:

# Specify the model name and revision
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
revision = "af0f99b"

# Load the model with the specified name and revision
# Instead of using pipeline, load the model directly
model = AutoModelForSequenceClassification.from_pretrained(model_name, revision=revision)
tokenizer = AutoTokenizer.from_pretrained(model_name, revision=revision)

# Wrap the model using the HuggingFaceModelWrapper
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

# Initialize the attack with the wrapped model
attack = textattack.attack_recipes.TextFoolerJin2019.build(model_wrapper)


# Generate adversarial examples for all the data given
original_texts = df['v2']  # Assuming 'df' is your DataFrame
adversarial_texts = []
successfully_attacked = []

for text in original_texts:
    try:
        attack_result = attack.attack([text])  # Ensure text is in a list
        adversarial_texts.append(attack_result[0].perturbed_text())
        successfully_attacked.append(True)
    except Exception as e:
        print(f"Failed to attack: {text}, Error: {e}")  # Log the error
        adversarial_texts.append(text)
        successfully_attacked.append(False)

# Combine original and adversarial examples
combined_texts = list(original_texts) + adversarial_texts
labels = [0] * len(original_texts) + [1] * len(adversarial_texts)  # 0 = non-adversarial, 1 = adversarial

# Create a DataFrame with the combined data
df_combined = pd.DataFrame({
    'text': combined_texts,
    'label': labels
})

# Specify the output folder path
output_folder = r"C:\Users\jonat\Desktop\Thesis papers\Thesis SPAM"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Construct the full file path
file_path = os.path.join(output_folder, 'adversarial_social_media_dataset.csv')

# Save the dataset to the specified folder
df_combined.to_csv(file_path, index=False)
print(f"Dataset saved successfully to: {file_path}")

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Streaming output truncated to the last 5000 lines.
Failed to attack: Waiting for your call., Error: Attack.attack() missing 1 required positional argument: 'ground_truth_output'
Failed to attack: Nope i waiting in sch 4 daddy... , Error: Attack.attack() missing 1 required positional argument: 'ground_truth_output'
Failed to attack: You have won ?1,000 cash or a ?2,000 prize! To claim, call09050000327, Error: Attack.attack() missing 1 required positional argument: 'ground_truth_output'
Failed to attack: I'm tired of arguing with you about this week after week. Do what you want and from now on, i'll do the same., Error: Attack.attack() missing 1 required positional argument: 'ground_truth_output'
Failed to attack: ÌÏ wait 4 me in sch i finish ard 5.., Error: Attack.attack() missing 1 required positional argument: 'ground_truth_output'
Failed to attack: our mobile number has won å£5000, to claim calls us back or ring the claims hot line on 09050005321., Error: Attack.attack() missing 1 re

Now the Attack on
##Targeted Sentiment Manipulation

In [18]:
!pip install --upgrade textattack

In [20]:
from textattack.attack_recipes import PSOZang2020
from transformers import AutoModelForSequenceClassification, AutoTokenizer


# Specify the model name and revision
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
revision = "af0f99b"

# Load the model with the specified name and revision
model = AutoModelForSequenceClassification.from_pretrained(model_name, revision=revision)
tokenizer = AutoTokenizer.from_pretrained(model_name, revision=revision)

# Wrap the model using the HuggingFaceModelWrapper
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

# Initialize the attack with the wrapped model
# We're using PSOZhang2020 as it's designed for sentiment manipulation
attack = PSOZang2020.build(model_wrapper)

# Generate adversarial examples for all the data given
original_texts = df['v2']  # Assuming 'df' is your DataFrame
adversarial_texts = []
successfully_attacked = []

for text in original_texts:
    try:
        # Set the target sentiment (0 for negative, 1 for positive)
        # You can adjust this based on your needs
        target_sentiment = 1 if model_wrapper.get_grad([text])[0] < 0.5 else 0

        attack_result = attack.attack(text, target_sentiment)
        if attack_result.successful:
            adversarial_texts.append(attack_result.perturbed_text)
            successfully_attacked.append(True)
        else:
            adversarial_texts.append(text)
            successfully_attacked.append(False)
    except Exception as e:
        print(f"Failed to attack: {text}, Error: {e}")  # Log the error
        adversarial_texts.append(text)
        successfully_attacked.append(False)

# Combine original and adversarial examples
combined_texts = list(original_texts) + adversarial_texts
labels = [0] * len(original_texts) + [1] * len(adversarial_texts)  # 0 = non-adversarial, 1 = adversarial

# Create a DataFrame with the combined data
df_combined = pd.DataFrame({
    'text': combined_texts,
    'label': labels
})

# Specify the output folder path
output_folder = r"C:\Users\jonat\Desktop\Thesis papers\Thesis SPAM"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Construct the full file path
file_path = os.path.join(output_folder, 'sentiment_manipulated_dataset.csv')

# Save the dataset to the specified folder
df_combined.to_csv(file_path, index=False)
print(f"Dataset saved successfully to: {file_path}")

textattack: Downloading https://textattack.s3.amazonaws.com/transformations/hownet/word_candidates_sense.pkl.
100%|██████████| 8.39M/8.39M [00:02<00:00, 4.04MB/s]
textattack: Copying /root/.cache/textattack/tmpyiqiou55.zip to /root/.cache/textattack/transformations/hownet/word_candidates_sense.pkl.
textattack: Successfully saved transformations/hownet/word_candidates_sense.pkl to cache.
textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Streaming output truncated to the last 5000 lines.
Failed to attack: Waiting for your call., Error: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]
Failed to attack: Nope i waiting in sch 4 daddy... , Error: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]
Failed to attack: You have won ?1,000 cash or a ?2,000 prize! To claim, call09050000327, Error: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]
Failed to attack: I'm tired of arguing with you about this week after week. Do what you want and from now on, i'll do the same., Error: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]
Failed to attack: ÌÏ wait 4 me in sch i finish ard 5.., Error: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]
Failed to attack: our mobile number has won å£5000, to claim calls us back or ring the claims hot line on 090500053